安裝必要模型

In [ ]:
# 1. 安裝 SAM2 與相關套件
!pip install -q git+https://github.com/facebookresearch/sam2.git
!apt-get install -y colmap

# 2. 下載 SAM2 模型權重
import os
if not os.path.exists("checkpoints"):
    os.makedirs("checkpoints")
!wget -q -P checkpoints https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_large.pt

# 3. 克隆 3DGS 預處理腳本 (用於將 COLMAP 轉為訓練格式)
if not os.path.exists("gaussian-splatting"):
    !git clone --recursive https://github.com/graphdeco-teresa/gaussian-splatting

生成mask

In [ ]:
import os
import cv2
import torch
import numpy as np
import shutil
from tqdm import tqdm
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor

from google.colab import drive
drive.mount('/content/drive')

# --- 設定路徑 ---
IMAGE_FOLDER = "/content/drive/MyDrive/Kim"
DATA_ROOT = "/content/data"
INPUT_DIR = os.path.join(DATA_ROOT, "input")
MASK_DIR = os.path.join(DATA_ROOT, "masks")

os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(MASK_DIR, exist_ok=True)

# --- 載入 SAM2 ---
device = torch.device("cuda")
predictor = SAM2ImagePredictor(build_sam2("sam2_hiera_l.yaml", "./checkpoints/sam2_hiera_large.pt", device=device))

# --- 執行處理 ---
image_files = [f for f in os.listdir(IMAGE_FOLDER) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
print(f"正在處理 {len(image_files)} 張圖片...")

for img_name in tqdm(image_files):
    img_path = os.path.join(IMAGE_FOLDER, img_name)
    img = cv2.imread(img_path)
    if img is None: continue

    # 複製原圖到 input
    shutil.copy(img_path, os.path.join(INPUT_DIR, img_name))

    # 生成 Mask
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    predictor.set_image(img_rgb)
    h, w = img_rgb.shape[:2]
    masks, _, _ = predictor.predict(
        point_coords=np.array([[w // 2, h // 2]]),
        point_labels=np.array([1]),
        multimask_output=False
    )

    # 存檔 (檔名需與原圖一致)
    mask_name = os.path.splitext(img_name)[0] + ".png"
    cv2.imwrite(os.path.join(MASK_DIR, mask_name), (masks[0] * 255).astype(np.uint8))

print("✅ Mask 生成與資料夾結構準備完成！")

跑COLMAP

In [ ]:
import os
import json

# 1. 執行 COLMAP (強制使用 CPU 進行 Sift 運算)
print("正在執行 COLMAP 特徵提取 (CPU 模式)...")
!colmap feature_extractor \
    --database_path /content/data/database.db \
    --image_path /content/data/input \
    --ImageReader.mask_path /content/data/masks \
    --ImageReader.single_camera 1 \
    --SiftExtraction.use_gpu 0

print("正在進行特徵比對 (CPU 模式)...")
!colmap exhaustive_matcher \
    --database_path /content/data/database.db \
    --SiftMatching.use_gpu 0

print("正在進行稀疏重建 (SfM)...")
!mkdir -p /content/data/sparse
!colmap mapper \
    --database_path /content/data/database.db \
    --image_path /content/data/input \
    --output_path /content/data/sparse

# 2. 轉換格式以產出文字檔
!mkdir -p /content/data/sparse/txt
!colmap model_converter \
    --input_path /content/data/sparse/0 \
    --output_path /content/data/sparse/txt \
    --output_type TXT

# 3. 產出給 Supersplat 使用的 cameras.json
def colmap_to_json(txt_path, output_json):
    if not os.path.exists(os.path.join(txt_path, "images.txt")):
        print("❌ 錯誤：重建失敗。請確保影片抽幀的照片有足夠重疊度。")
        return

    cameras_dict = {}
    with open(os.path.join(txt_path, "cameras.txt"), "r") as f:
        for line in f:
            if line.startswith("#") or not line.strip(): continue
            p = line.split()
            cameras_dict[p[0]] = {"width": int(p[2]), "height": int(p[3]), "params": [float(x) for x in p[4:]]}

    frames = []
    with open(os.path.join(txt_path, "images.txt"), "r") as f:
        lines = [l.strip() for l in f.readlines() if l.strip() and not l.startswith("#")]
        for i in range(0, len(lines), 2):
            parts = lines[i].split()
            cam_id = parts[8]
            frames.append({
                "file_path": parts[-1],
                "rotation": [float(parts[1]), float(parts[2]), float(parts[3]), float(parts[4])], # QW, QX, QY, QZ
                "translation": [float(parts[5]), float(parts[6]), float(parts[7])],
                "intrinsics": cameras_dict.get(cam_id)
            })

    with open(output_json, 'w') as f:
        json.dump(frames, f, indent=4)
    print(f"✅ 成功導出相機角度 JSON：{output_json}")

if os.path.exists("/content/data/sparse/txt"):
    colmap_to_json("/content/data/sparse/txt", "/content/data/cameras.json")

打包下載

In [ ]:
import os
import shutil
from google.colab import files

# 設定輸出檔名與路徑
output_zip = "/content/masks_and_camera.zip"
mask_folder = "/content/data/masks"  # 確認這是存放 Mask 的資料夾路徑
camera_json = "/content/data/cameras.json"

# 檢查資料是否存在
if os.path.exists(mask_folder) and os.path.exists(camera_json):
    print("📦 正在打包 Mask 影像與 Camera JSON...")

    # -j: 不保留目錄結構(放同一層), -r: 遞迴處理資料夾
    # 如果您希望 zip 裡面維持 masks/ 資料夾結構，把 -j 去掉
    !zip -r {output_zip} {mask_folder} {camera_json}

    print("🎉 打包完成！")
    files.download(output_zip)
else:
    if not os.path.exists(mask_folder):
        print(f"❌ 找不到 Mask 資料夾: {mask_folder}")
    if not os.path.exists(camera_json):
        print(f"❌ 找不到相機檔案: {camera_json}")